In [6]:
%load_ext autoreload
%autoreload 2
#from raxutil.nlp.faresheet import *
#from raxutil.nlp.production import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import cssutils
from functools import reduce
import os
from os.path import basename
import csv, ast
from glob import glob
from collections import Counter, defaultdict
from nltk.classify import MaxentClassifier 

import pymongo
from pymongo import MongoClient
from pymongo import TEXT
pd.options.display.max_rows = 99999

In [8]:
client = MongoClient('localhost:27017')
db=client.raxdb
faresheet = 'CXfaresheets_new'
faresheet_old = 'CXfaresheets' 
fs = db['CXfaresheets_new']

### Identify different templates of US commission sheets

In [448]:
us_docs = [doc for doc in fs.find({'country': 'US'}, 
                                  {'predictions':1, 'classifications':1, 'filename':1, 'country':1})]

In [450]:
len(us_docs)

1405

In [461]:
us_df = pd.DataFrame(us_docs)

In [1]:
for index, row in us_df.iterrows():
    try:
        us_df.loc[index, 'gold'] = row['classifications']['Commission']
    except:
        print("Can't get commission gold: ", row)
    us_df.loc[index, 'pred'] = row['predictions']['Commission']

In [482]:
# get all the predicted commission sheet
mask1 = us_df['gold']=='yes'
mask2 = us_df['pred']=='yes'
us_comm = us_df.loc[mask1|mask2]
mask3 = us_df['gold']!='no'
us_comm = us_comm.loc[mask3]
us_comm.drop(['classifications', 'predictions'], axis=1, inplace=True)

In [492]:
for index, row in us_comm.iterrows():
    text = fs.find_one({'_id': row['_id']}, {'teststring':1})
    us_comm.loc[index, 'teststring'] = text['teststring']

In [498]:
# feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=3, lowercase=False,
                                   stop_words='english', analyzer='word')
tfidf = tfidf_vectorizer.fit_transform(us_comm['teststring'])
tfidf_feature_list = tfidf_vectorizer.get_feature_names()

In [499]:
len(tfidf_feature_list)

7702

In [512]:
from sklearn.cluster import KMeans
km_model = KMeans(n_clusters=2)
km_model.fit(tfidf)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [511]:
tfidf.shape

(319, 7702)

In [514]:
us_comm['cluster'] = km_model.labels_

In [517]:
us_comm.to_csv('/Users/chenjialu/Desktop/commission_files.csv')

In [530]:
us_comm.sort_values(by=['cluster', 'filename'], inplace=True)

In [28]:
pd.options.display.max_colwidth = 100 
us_comm[['filename', 'cluster']].loc[us_comm['cluster']==1].to_csv('/Users/chenjialu/Desktop/commission_templateA.csv')

In [4]:
us_comm = pd.read_csv('/Users/chenjialu/Desktop/commission_files.csv')

In [14]:
# take cluster 0 to further split 
us_comm2 = us_comm.loc[us_comm['cluster']==0]

In [15]:
# feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=10, lowercase=False,
                                   stop_words='english', analyzer='word')
tfidf = tfidf_vectorizer.fit_transform(us_comm2['teststring'])
tfidf_feature_list = tfidf_vectorizer.get_feature_names()

In [16]:
len(tfidf_feature_list)

1980

In [18]:
tfidf.shape

(127, 1980)

In [34]:
from sklearn.cluster import KMeans
km_model = KMeans(n_clusters=4)
km_model.fit(tfidf)
clustering = collections.defaultdict(list)
for idx, label in enumerate(km_model.labels_):
        clustering[label].append((us_comm2.iloc[idx]['filename'], us_comm2.iloc[idx]['country'])) 

In [35]:
# labels = [3 if i==1 else 0 for i in km_model.labels_]
us_comm2['cluster'] = km_model.labels_

/anaconda3/envs/rassure_nltk/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [36]:
us_comm2.sort_values(by=['cluster', 'filename'], inplace =True)

/anaconda3/envs/rassure_nltk/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [44]:
us_comm2['template'] = ''

/anaconda3/envs/rassure_nltk/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [46]:
# mark incentive agreement as template A
us_comm2.loc[us_comm2['teststring'].str.contains('INCENTIVE AGREEMENT'), 'template'] = 'A'

/anaconda3/envs/rassure_nltk/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [49]:
us_comm2[['filename', 'cluster', 'template']].to_csv('/Users/chenjialu/Desktop/other_templates.csv')

### Add file path in the check list:

In [27]:
us_comm = pd.read_csv('/Users/chenjialu/Desktop/CX_Commissions/Predicted Commissions/US/commission_check_US1.csv')

In [28]:
us_comm['filepath']=''
us_comm['manual check']=''
us_comm['human prediction']=''
us_comm['model prediction']=''

In [30]:
columns = ['filename', 'cluster', 'template', 'filepath', 'human prediction', 'model prediction', 'manual check']
us_comm = us_comm[columns]

In [31]:
us_comm.to_csv('/Users/chenjialu/Desktop/CX_Commissions/Predicted Commissions/US/commission_check_US1.csv', index=False)

### Functions

In [14]:
def get_market_comm(db, faresheet, country):
    us_docs = [doc for doc in db[faresheet].find({'country': country}, 
                                  {'predictions':1, 'classifications':1, 'path':1, 'teststring':1, 'filename':1, 'country':1})]
    us_df = pd.DataFrame(us_docs)
    us_df['gold'] = ''
    # labelling
    for index, row in us_df.iterrows():
        try:
            us_df.loc[index, 'gold'] = row['classifications']['Commission']
        except:
            print("Can't get commission gold: ", row['filename'])
        us_df.loc[index, 'pred'] = row['predictions']['Commission']
        
    # get all the predicted/classified commission sheet
    mask1 = us_df['gold']=='yes'
    mask2 = us_df['pred']=='yes'
    us_comm = us_df.loc[mask1|mask2]
    mask3 = us_df['gold']!='no'
    us_comm = us_comm.loc[mask3]
    us_comm.drop(['classifications', 'predictions'], axis=1, inplace=True)
    
    return us_comm

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
def tfidf_cluster(us_comm, k=2, v_min_df=2):
    tfidf_vectorizer = TfidfVectorizer(min_df=v_min_df, lowercase=False,
                                   stop_words='english', analyzer='word')
    tfidf = tfidf_vectorizer.fit_transform(us_comm['teststring'])
    tfidf_feature_list = tfidf_vectorizer.get_feature_names()
    print("Number of features: ", len(tfidf_feature_list))
    km_model = KMeans(n_clusters=k)
    km_model.fit(tfidf)
    us_comm['cluster'] = km_model.labels_
    us_comm.sort_values(['cluster', 'filename'], inplace=True)
    return us_comm

In [16]:
from sklearn.metrics import silhouette_score
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
def choose_k(dat_comm, k1=2, k2=11, v_min_df=2):
    """A function to choose k with tfidf, A higher Silhouette Coefficient indicates that the object is well matched to 
    its own cluster and poorly matched to neighboring clusters."""
    tfidf_vectorizer = TfidfVectorizer(min_df=v_min_df, lowercase=False,
                                   stop_words='english', analyzer='word')
    tfidf = tfidf_vectorizer.fit_transform(dat_comm['teststring'])
    tfidf_feature_list = tfidf_vectorizer.get_feature_names()
    print("Number of features: ", len(tfidf_feature_list))
    for n_cluster in range(k1, k2):
        kmeans = KMeans(n_clusters=n_cluster).fit(tfidf)
        label = kmeans.labels_
        sil_coeff = silhouette_score(tfidf, label, metric='euclidean')
        print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

In [4]:
def create_checklist(us_comm, country):
    new_comm = us_comm.copy()
    # add filepath
    for index, row in us_comm.iterrows():
        us_doc = [doc for doc in fs.find({'filename': row['filename'], 'country': country}, 
                                   {'path':1, 'predictions':1, 'classifications':1} )][0]        
        new_comm.loc[index, 'filepath'] = us_doc['path']
#         try:
#             us_comm.loc[index, 'human prediction'] = us_doc['classifications']['Commission']
#         except:
#             print("no human classification for: ", row['filename'])
        #us_comm.loc[index, 'model prediction'] = us_doc['predictions']['Commission']
    return new_comm
    
# au_comm=create_checklist(au_comm, 'AU') 

### Production Example for a market:

In [5]:
nz_comm = get_market_comm(db, faresheet, 'NZ')

In [109]:
choose_k(nz_comm, k1=2, k2=10)

For n_clusters=2, The Silhouette Coefficient is 0.4538953197427514
For n_clusters=3, The Silhouette Coefficient is 0.44097637723692046
For n_clusters=4, The Silhouette Coefficient is 0.4745091049347023
For n_clusters=5, The Silhouette Coefficient is 0.5151229959523871
For n_clusters=6, The Silhouette Coefficient is 0.5409860794471184
For n_clusters=7, The Silhouette Coefficient is 0.5656425679343441
For n_clusters=8, The Silhouette Coefficient is 0.5840518082361282
For n_clusters=9, The Silhouette Coefficient is 0.5657587513455863


choose k as 4

In [115]:
nz_comm=tfidf_cluster(nz_comm, k=4, v_min_df=3)

Number of features:  1134


In [120]:
nz_comm[['filename', 'pred', 'gold', 'cluster', 'path']].to_csv('/Users/chenjialu/Desktop/commission_check.csv', index=False)